In [2]:
"""
아시아타임즈 ‘[은행가소식]’ 기사 크롤러 (통합 버전)
- 목록: https://www.asiatime.co.kr/list/31?page=N
- 제목에 [은행가소식] 이 포함된 기사만 수집
- 날짜 파싱: meta → <time> → ‘입력 :’·‘등록 :’ 정규식 3단계
- 본문 파싱: 광고·SNS·저작권 단락 제외하고 20자 이상만 모아 합침
"""

import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ────────────────────── 기본 설정 ──────────────────────
BASE_URL  = "https://www.asiatime.co.kr"
LIST_PATH = "/list/31"                          # ‘은행’ 카테고리 ID
HEADERS   = {"User-Agent": "Mozilla/5.0 (compatible; BankerBot/1.0)"}
BANKER_RX = re.compile(r"\[은행가소식]")

# ────────────────────── 공통 유틸 ──────────────────────
def fetch(url: str, timeout: int = 10) -> BeautifulSoup:
    """URL → BeautifulSoup 객체 반환"""
    res = requests.get(url, headers=HEADERS, timeout=timeout)
    res.raise_for_status()
    return BeautifulSoup(res.text, "lxml")

# ────────────────────── 날짜 파서 ──────────────────────
def parse_date(soup: BeautifulSoup) -> str:
    """meta → <time> → ‘입력/등록 :’ 정규식 순으로 날짜·시간 추출"""
    META_KEYS = [
        ("property", "article:published_time"),
        ("property", "og:article:published_time"),
        ("name",     "pubdate"),
        ("name",     "publish-date"),
        ("name",     "date"),
    ]
    # ① meta 태그
    for attr, key in META_KEYS:
        m = soup.find("meta", {attr: key})
        if m and m.get("content"):
            ts = m["content"].strip().split()
            if len(ts) >= 2:
                return ts[0] + " " + ts[1][:5]

    # ② <time> 태그
    t = soup.find("time")
    if t:
        if t.get("datetime"):
            return t["datetime"][:16]
        txt = t.get_text(" ", strip=True)
        if re.search(r"\d{4}[-./]\d{2}[-./]\d{2}", txt):
            return txt.strip()

    # ③ 본문 텍스트 패턴
    m = re.search(
        r"(입력|등록|게재)\s*[:·]?\s*(\d{4}[-./]\d{2}[-./]\d{2}\s*\d{2}:\d{2})",
        soup.get_text(" ", strip=True)
    )
    return m.group(2) if m else "Unknown"

# ────────────────────── 본문 파서 ──────────────────────
AD_CLASSES  = {"pc_article_ad_01", "m_article_ad_02"}
AD_PATTERNS = re.compile(r"advertisement|sns|무단전재|저작권", re.I)

def parse_body(soup: BeautifulSoup) -> str:
    """
    - .article-body 또는 #article-view-content-div → 없으면 soup 전체
    - <p>, <figcaption> 중 길이 20자↑ & 광고 키워드 없는 것만 추출
    """
    container = (
        soup.select_one(".article-body")
        or soup.select_one("#article-view-content-div")
        or soup
    )

    paras = []
    for tag in container.find_all(["p", "figcaption"], recursive=True):
        # 광고 영역 자체 또는 그 내부는 제외
        if tag.find_parent(class_=lambda c: c in AD_CLASSES if c else False):
            continue
        if any(c in AD_CLASSES for c in (tag.get("class") or [])):
            continue

        text = tag.get_text(" ", strip=True).replace("\xa0", " ").strip()
        if len(text) < 20:
            continue
        if AD_PATTERNS.search(text):
            continue

        paras.append(text)

    return "\n".join(paras)

# ────────────────────── 기사 파서 ──────────────────────
def parse_article(url: str) -> dict:
    soup = fetch(url)
    title_tag = soup.find("h1") or soup.find("h2")
    title = title_tag.get_text(" ", strip=True) if title_tag else "No title"
    author_tag = soup.select_one(".writer, .author")
    author = author_tag.get_text(strip=True) if author_tag else "Unknown"

    return {
        "url":     url,
        "title":   title,
        "date":    parse_date(soup),
        "author":  author,
        "content": parse_body(soup),
    }

# ────────────────────── 목록 파서 ──────────────────────
def banker_links(list_soup: BeautifulSoup) -> list[str]:
    links = []
    for a in list_soup.select("a[href^='/article/']"):
        if BANKER_RX.search(a.get_text(strip=True)):
            links.append(urljoin(BASE_URL, a["href"].split("?")[0]))
    return links

# ────────────────────── 크롤러 진입점 ──────────────────────
def crawl_banker_news(max_pages: int = 10, delay: float = 1.0) -> pd.DataFrame:
    rows, seen = [], set()
    for page in range(1, max_pages + 1):
        list_url = f"{BASE_URL}{LIST_PATH}?page={page}"
        list_soup = fetch(list_url)
        links = banker_links(list_soup)

        # [은행가소식]이 더 이상 없으면 조기 종료
        if not links:
            break

        for link in links:
            if link in seen:
                continue
            try:
                rows.append(parse_article(link))
            except Exception as e:
                print(f"[warn] {link} skipped: {e}")
            seen.add(link)
            time.sleep(delay)                      # 서버 부하 방지

    return pd.DataFrame(rows)

# ────────────────────── 실행 예시 ──────────────────────
if __name__ == "__main__":
    df = crawl_banker_news(max_pages=10, delay=1.0)
    df.to_csv("asiatime_banker_news.csv", index=False, encoding="utf-8-sig")
    print(f"Saved {len(df)} articles → asiatime_banker_news.csv")


Saved 81 articles → asiatime_banker_news.csv


In [7]:
import pandas as pd

# 1) 크롤러가 만든 파일 로드
df = pd.read_csv("asiatime_banker_news.csv")

# 2) '\n□' 로 분리 → 리스트를 새 열(section)에 담고 explode로 길게 변환
df["section"] = df["content"].str.split(r"\n□\s*")       # \s* = 기호 뒤 공백 무시
long_df = df.explode("section", ignore_index=True)

# 3) 공백·빈 문자열 제거
long_df["section"] = long_df["section"].str.strip()
long_df = long_df[long_df["section"] != ""]

# 4) 필요한 컬럼만 남기고 한글 이름으로 바꿔 주기
long_df = (
    long_df.rename(columns={"date": "작성일", "url": "링크", "section": "내용"})
           .loc[:, ["작성일", "링크", "내용"]]
)

# 5) 저장
long_df.to_csv("asiatime_banker_sections.csv", index=False, encoding="utf-8-sig")
print(f"{len(long_df):,}개 단락 저장 완료 → asiatime_banker_sections.csv")

463개 단락 저장 완료 → asiatime_banker_sections.csv


In [8]:
long_df

,작성일,링크,내용
0,2025-05-22 18:37,https://www.asiatime.co.kr/article/20250522500400,유승열 기자 입력 2025-05-22 18:37 수정 2025-05-22 18:37
1,2025-05-22 18:37,https://www.asiatime.co.kr/article/20250522500400,은행권 공동 본인확인서비스 MOU 체결\n국민·농협·신한·우리·하나·기업은행은 21...
2,2025-05-22 18:37,https://www.asiatime.co.kr/article/20250522500400,은행연합회 유럽 금융기관과 협력강화\n은행연합회는 5월 20일부터 21일까지 독일과...
3,2025-05-22 18:37,https://www.asiatime.co.kr/article/20250522500400,"KB금융공익재단, 해군 재경근무지원대대 장병 대상 경제·금융교육 실시\nKB금융공익..."
4,2025-05-22 18:37,https://www.asiatime.co.kr/article/20250522500400,"신한은행 노란우산 가입 소상공인 금융지원 실시\n신한은행은 22일 중소기업중앙회, ..."
...,...,...,...
458,2025-03-20 17:53,https://www.asiatime.co.kr/article/20250320500402,정종진 기자 입력 2025-03-20 17:53 수정 2025-03-20 17:53
459,2025-03-20 17:53,https://www.asiatime.co.kr/article/20250320500402,전북은행 '봄날 특판 예금' 출시\n전북은행이 봄맞이 이벤트 우대금리를 제공하는 '...
460,2025-03-20 17:53,https://www.asiatime.co.kr/article/20250320500402,토스뱅크 '수출 소상공인 지원' 무보와 맞손\n토스뱅크가 한국무역보험공사와 협력해 ...
461,2025-03-20 17:53,https://www.asiatime.co.kr/article/20250320500402,농협은행 '우리 쌀 꾸러미' 전달\n농협은행 기업금융부문이 19일 경기도 화성시 소...


In [9]:
import pandas as pd, re

JUNK = re.compile(r"기자\s*입력|입력\s*[:·]|수정\s*[:·]|Copyright|All rights reserved"
                  r"|건전한\s*토론문화|고충처리인|뉴스제휴위원회"
                  r"|\bTEL\b|\bemail\b", re.I)

df = pd.read_csv("asiatime_banker_sections.csv")
df["내용"] = df["내용"].str.replace(JUNK, "", regex=True).str.strip()
df = df[df["내용"].str.len() > 0]        # 완전히 빈 행 삭제
df.to_csv("asiatime_banker_sections_clean.csv", index=False, encoding="utf-8-sig")
print("잡문 제거 완료 → asiatime_banker_sections_clean.csv")

잡문 제거 완료 → asiatime_banker_sections_clean.csv


In [13]:
"""
아시아타임즈 ‘[은행가소식]’ 크롤러 (footer 제거‧news-only 버전)
"""

import re, time, requests, pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL  = "https://www.asiatime.co.kr"
LIST_PATH = "/list/31"                       # ‘은행’ 카테고리
HEADERS   = {"User-Agent": "Mozilla/5.0 (compatible; BankerBot/2.0)"}
BANKER_RX = re.compile(r"\[은행가소식]")

# ─────────────────────── fetch ────────────────────────
def fetch(url: str, timeout: int = 10) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return BeautifulSoup(r.text, "lxml")

# ─────────────────────── 날짜 ─────────────────────────
def parse_date(soup: BeautifulSoup) -> str:
    for meta_key in [
        ("property", "article:published_time"),
        ("property", "og:article:published_time"),
        ("name",     "pubdate"), ("name", "publish-date"), ("name", "date")
    ]:
        m = soup.find("meta", {meta_key[0]: meta_key[1]})
        if m and m.get("content"):
            ts = m["content"].split()
            if len(ts) >= 2:
                return ts[0] + " " + ts[1][:5]

    t = soup.find("time")
    if t:
        return (t.get("datetime") or t.get_text()).strip()[:16]

    m = re.search(r"(입력|등록)\s*[:·]?\s*(\d{4}-\d{2}-\d{2} \d{2}:\d{2})",
                  soup.get_text(" ", strip=True))
    return m.group(2) if m else "Unknown"

# ─────────────────────── 본문 ─────────────────────────
# 1) 기사 본문이 시작되기 전에 흔히 보이는 패턴(기자·입력·수정)
HEAD_JUNK_RX = re.compile(
    r"(기자\s*입력|입력\s*[:·]|수정\s*[:·]|데스크\s*:)", re.I
)

# 2) 본문이 끝나면 나타나는 푸터(댓글 안내·저작권·고충처리인 …)
TAIL_JUNK_RX = re.compile(
    r"Copyright|All\s+rights\s+reserved|건전한\s*토론문화|고충처리인"
    r"|뉴스제휴위원회|\bTEL\b|\bemail\b", re.I
)

def parse_body(soup: BeautifulSoup) -> str:
    """
    ① 본문 container(.article-body / #article-view-content-div / fallback soup)
    ② <p>,<figcaption> 순서대로 훑으면서
       · HEAD_JUNK_RX가 처음 등장하기 전 문단 skip
       · TAIL_JUNK_RX 만나면 break
       · 20자 이상 & 광고·SNS 문구 없는 것만 저장
    """
    container = (
        soup.select_one(".article-body")
        or soup.select_one("#article-view-content-div")
        or soup
    )

    AD_CLASSES  = {"pc_article_ad_01", "m_article_ad_02"}
    AD_RX       = re.compile(r"advertisement|sns|무단전재|저작권", re.I)

    paras, started = [], False

    for tag in container.find_all(["p", "figcaption"], recursive=True):
        # 광고 블록 무시
        if tag.find_parent(class_=lambda c: c in AD_CLASSES if c else False):
            continue
        if any(c in AD_CLASSES for c in (tag.get("class") or [])):
            continue

        text = tag.get_text(" ", strip=True).replace("\xa0", " ").strip()

        if not text or len(text) < 20:
            continue

        # 머리말(기자·입력 정보) 스킵
        if not started:
            if HEAD_JUNK_RX.search(text):
                continue
            started = True                       # 여기부터 본문
        # 본문 끝 지점이면 종료
        if TAIL_JUNK_RX.search(text):
            break
        # 광고·SNS 안내 같은 노이즈 제거
        if AD_RX.search(text):
            continue

        paras.append(text)

    return "\n".join(paras)

# ─────────────────────── 기사 파서 ────────────────────
def parse_article(url: str) -> dict:
    soup = fetch(url)
    title_tag = soup.find("h1") or soup.find("h2")
    return {
        "url":   url,
        "date":  parse_date(soup),
        "content": parse_body(soup),
    }

# ─────────────────────── 목록 파서 ────────────────────
def banker_links(list_soup: BeautifulSoup) -> list[str]:
    links = []
    for a in list_soup.select("a[href^='/article/']"):
        if BANKER_RX.search(a.get_text(strip=True)):
            links.append(urljoin(BASE_URL, a["href"].split("?")[0]))
    return links

# ─────────────────────── 크롤링 ───────────────────────
def crawl_banker(max_pages: int = 10, delay: float = 1.0) -> pd.DataFrame:
    rows, seen = [], set()
    for page in range(1, max_pages + 1):
        soup = fetch(f"{BASE_URL}{LIST_PATH}?page={page}")
        links = banker_links(soup)
        if not links:
            break

        for link in links:
            if link in seen:
                continue
            try:
                art = parse_article(link)
                if art["content"]:               # 빈 기사 거르기
                    rows.append(art)
            except Exception as e:
                print(f"[warn] {link} skipped: {e}")
            seen.add(link)
            time.sleep(delay)
    return pd.DataFrame(rows)

# ─────────────────────── 실행 예시 ────────────────────
if __name__ == "__main__":
    df = crawl_banker(max_pages=10, delay=1.0)
    df.to_csv("asiatime_banker_news_clean.csv", index=False, encoding="utf-8-sig")
    print(f"{len(df):,}건 저장 → asiatime_banker_news_clean.csv")


81건 저장 → asiatime_banker_news_clean.csv
